In [9]:
# generate csv from data 

import csv

header = ['phrase_id', 'phrase', 'sentiment_score', 'implication']

f_dict = open('./data/english/dictionary.txt', 'r')
f_sent = open('./data/english/sentiment_labels.txt', 'r')

f = open('./data/english/sentiment_data.csv', 'w')
writer = csv.writer(f)
writer.writerow(header)

lines_dict = f_dict.readlines()
lines_sent = f_sent.readlines()

for line in lines_dict:

    data = []
    line_dict_list = line.strip().split('|')
    sentiment_val = lines_sent[int(line_dict_list[1])+1].split('|')[1]
    sentiment_val = float(sentiment_val)
    implication = ""

    if sentiment_val<=0.2:
        implication = "very_negative"
    elif sentiment_val<=0.4:
        implication = "negative"
    elif sentiment_val<=0.6:
        implication = "neutral"
    elif sentiment_val<=0.8:
        implication = "positive"
    else:
        implication = "very_positive"

    data.append(line_dict_list[1])
    data.append(line_dict_list[0])
    data.append(sentiment_val)
    data.append(implication)

    writer.writerow(data)

f_dict.close()
f_sent.close()
f.close()


In [13]:
# remove urls
import pandas as pd  

df = pd.read_csv('./data/english/sentiment_data.csv')    

df['phrase'] = df['phrase'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

df.to_csv('./data/english/sentiment_data_url_removed.csv')


In [7]:
# remove stop words
# https://stackabuse.com/python-for-nlp-working-with-the-gensim-library-part-1/

from gensim.parsing.preprocessing import remove_stopwords
import pandas as pd  

df = pd.read_csv('./data/english/sentiment_data_url_removed.csv')

for index, phrase in df['phrase'].iteritems():
    df['phrase'][index] = remove_stopwords(phrase)

df.to_csv('./data/english/sentiment_data_url_stopwords_removed.csv')



In [8]:
# lowercase all words
import pandas as pd  

df = pd.read_csv('./data/english/sentiment_data_url_removed.csv')

df['phrase'] = [each_string.lower() for each_string in df['phrase']]

df.to_csv('./data/english/sentiment_data_url_stopwords_removed_lower.csv')


In [ ]:
# stemming

import pandas as pd  

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
ps = PorterStemmer()

df = pd.read_csv('./data/english/sentiment_data_url_stopwords_removed_lower.csv')
ind = 0

for each_string in df['phrase']:
    words = word_tokenize(each_string)
    final_string = ''
    
    for w in words:
        final_string += ps.stem(w)
        final_string += ' '

    df['phrase'][ind]=final_string
    ind=ind+1

df.to_csv('./data/english/sentiment_data_url_stopwords_removed_lower_stemming.csv')


In [ ]:
# lematization
import pandas as pd  

import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

df = pd.read_csv('./data/english/sentiment_data_url_stopwords_removed_lower_stemming.csv')
ind = 0

for each_string in df['phrase']:
    words = str(each_string).split(' ')
    final_string = ''

    for w in words:
        final_string += lemmatizer.lemmatize(w)
        final_string += ' '

    df['phrase'][ind]=final_string
    ind=ind+1

df.to_csv('./data/english/sentiment_data_url_stopwords_removed_lower_stemming_lematization.csv')


In [4]:
import pandas as pd

df = pd.read_csv('./data/english/sentiment_data_url_stopwords_removed_lower_stemming_lematization.csv', usecols = [4,5,6,7])
df.to_csv('./data/english/sentiment_data_preprocessed.csv',index=False)


In [16]:
# Label Encoding and Tokenisation
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 

# Label Encoding
lb=LabelEncoder()
df['implication'] = lb.fit_transform(df['implication'])

# Tokenisation
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(df['phrase'].values)
X = tokenizer.texts_to_sequences(df['phrase'].values)
X = pad_sequences(X)


In [23]:
# Creating LSTM Model

model = Sequential()
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(176, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())


NotImplementedError: Cannot convert a symbolic Tensor (lstm_10/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported